In [14]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 
# import talib as ta
import math
# from numba import jit

from importlib import reload 
from qflib import basic
reload(basic)
global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

In [15]:
# 指标库读出基本数据 
xl_01dgree = 0.001745 # 0.1度 
xl_02dgree = 0.003490 # 0.2度 
xl_03dgree = 0.005230 # 0.3度
xl_dgree = xl_03dgree
print( 'd60 斜率：', str(xl_dgree) )

sql = "SELECT * \
from day_zhibiao \
where \
jx_days_ud60 > 20 and jx_days_ud60 < 40 and \
jx_xl_60>-"+str(xl_dgree)+" and jx_xl_10>0 and \
lj_fl_1_3 > 1.1 and lj_fl_3_10>1.05 "
print(sql)
df = pd.read_sql_query(sql, conn)

df.shape

d60 斜率： 0.00523
SELECT * from day_zhibiao where jx_days_ud60 > 20 and jx_days_ud60 < 40 and jx_xl_60>-0.00523 and jx_xl_10>0 and lj_fl_1_3 > 1.1 and lj_fl_3_10>1.05 


(35170, 80)

In [16]:
# 检测原数据
df.head(10)

,ts_code,symbol,trade_date,open,high,low,close,pre_close,change,pct_change,...,jx_dg_60,jx_dg_120,jx_dg_250,pct_fl_1_3,pct_fl_3_10,pct_fl_5_20,pct_fl_5_60,pct_fl_20_60,vol_ma120,vol_ma250
0,0,000001,2023-04-18,12.65,12.92,12.61,12.72,12.65,0.07,0.55,...,-0.149989,0.071556,-0.056115,1.18279,1.83067,1.672700,1.254040,0.800644,1165250.0,1098560.0
1,0,000001,2023-04-14,12.32,12.62,12.30,12.41,12.28,0.13,1.06,...,-0.142045,0.052575,-0.061307,1.27158,1.60911,1.149490,0.945933,0.800511,1154620.0,1098100.0
2,0,000001,2023-04-13,12.13,12.39,12.02,12.28,12.20,0.08,0.66,...,-0.150996,0.040595,-0.052240,1.34096,1.34546,0.958903,0.808466,0.788089,1151510.0,1102860.0
3,0,000001,2022-08-19,11.99,12.47,11.95,12.29,11.97,0.32,2.67,...,-0.100935,-0.106905,-0.090876,1.60905,1.89532,1.527590,1.151970,0.781507,1118530.0,1116760.0
4,0,000001,2022-08-18,12.20,12.26,11.88,11.97,12.12,-0.15,-1.24,...,-0.138451,-0.127292,-0.077977,1.28270,1.27416,1.175690,0.922218,0.757611,1112010.0,1115350.0
5,0,000001,2022-08-17,11.87,12.14,11.79,12.12,11.85,0.27,2.28,...,-0.127994,-0.123245,-0.073095,1.49566,1.36661,1.155420,0.957206,0.808617,1108820.0,1112540.0
6,0,000001,2022-06-15,13.59,14.60,13.55,14.23,13.61,0.62,4.56,...,0.070022,-0.110836,-0.121417,2.52932,1.58920,1.938680,1.705390,1.039200,1152950.0,1056780.0
7,0,000001,2022-01-20,15.96,16.95,15.91,16.82,15.99,0.83,5.19,...,-0.144305,-0.039488,-0.033062,2.83109,1.30102,1.427690,1.577360,1.273010,1108120.0,994441.0
8,0,000001,2021-08-20,19.46,19.56,18.19,18.91,19.83,-0.92,-4.64,...,-0.245854,-0.040030,0.061569,1.62871,1.30429,0.939935,1.233120,1.463400,778422.0,976405.0
9,0,000002,2022-08-31,16.38,16.75,16.19,16.63,15.89,0.74,4.66,...,0.003986,-0.001669,-0.054032,2.53564,1.52304,1.471540,1.125890,0.786059,1068560.0,1062830.0


In [17]:
""" 数据抽查 sz_max_50 """
sum = len(df)
bins = [-float('inf'), 0, 10, float('inf')] 
labels = ['=0', '0-10', '>10']
df['sz_max_5_bin'] = pd.cut(df['sz_max_5'], bins=bins, labels=labels)
df['sz_max_5_bin'].value_counts(), df['sz_max_5_bin'].value_counts()/sum

(0-10    22639
 =0      10041
 >10      2490
 Name: sz_max_5_bin, dtype: int64,
 0-10    0.643702
 =0      0.285499
 >10     0.070799
 Name: sz_max_5_bin, dtype: float64)

In [18]:
""" 数据抽查 sz_max_10 """
sum = len(df)
bins = [-float('inf'), 5, 12, float('inf')] 
labels = ['0-5', '5-12', '>12']
df['sz_max_10_bin'] = pd.cut(df['sz_max_10'], bins=bins, labels=labels)
df['sz_max_10_bin'].value_counts(), df['sz_max_10_bin'].value_counts()/sum

(0-5     22561
 5-12     8740
 >12      3869
 Name: sz_max_10_bin, dtype: int64,
 0-5     0.641484
 5-12    0.248507
 >12     0.110009
 Name: sz_max_10_bin, dtype: float64)

In [19]:
""" 数据抽查 sz_max_20 """
sum = len(df)
bins = [-float('inf'), 5, 12, float('inf')] 
labels = ['<=5', '5-12', '>12']
df['sz_max_20_bin'] = pd.cut(df['sz_max_20'], bins=bins, labels=labels)
df['sz_max_20_bin'].value_counts(), df['sz_max_20_bin'].value_counts()/sum

(<=5     16385
 5-12    10731
 >12      8054
 Name: sz_max_20_bin, dtype: int64,
 <=5     0.465880
 5-12    0.305118
 >12     0.229002
 Name: sz_max_20_bin, dtype: float64)

In [20]:
# 涨幅阶梯化

def jieti_10(df):
    # rate = 0, 1, 2, 3, 4, 5
    if df['sz_max_10']<=5: rate = 0        
    if df['sz_max_10']>5 and df['sz_max_10']<=12: rate = 1          
    if df['sz_max_10']>12: rate = 2        
    return rate
def jieti_20(df):
    # rate = 0, 1, 2
    if df['sz_max_20'] <= 5: rate = 0        
    if df['sz_max_20']>5 and df['sz_max_10']<=12: rate = 1        
    # if df['sz_max_20']>10 and df['sz_max_10']<=20: rate = 2        
    if df['sz_max_20']>12: rate = 2        
    return rate

df['sz_jt_10'] = df.apply(jieti_10, axis=1) 
df['sz_jt_20'] = df.apply(jieti_20, axis=1) 

In [21]:
# 数据监测
df[ ['sz_max_10', 'sz_jt_10']].head(20)
# df[ ['sz_max_20', 'sz_jt_20']].head(20)

,sz_max_10,sz_jt_10
0,0.000000,0
1,2.497990,0
2,3.583060,0
3,1.464610,0
4,4.177110,0
5,2.887790,0
6,0.000000,0
7,0.118906,0
8,0.000000,0
9,11.244700,1


In [28]:
""" 形成数据集 """

# 0-1类训练目标s：
df['test00'] = df['sz_max_20'] > 10
df['test5_8'] = df['sz_max_5'] > 8
df['test10_10'] = df['sz_max_10'] > 10
df['test20_10'] = df['sz_max_20'] > 10
df['test20_20'] = df['sz_max_20'] > 20
df['test20_30'] = df['sz_max_20'] > 30
# 多分类训练目标
df['class0'] = df['sz_jt_10']
# 确定输出列  
col_list_ds = ['ts_code', 'symbol', 'trade_date', 'pct_change',  \
    'diff', 'dea', 'bar',\
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', 'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
    'sz_max_5','sz_max_10','sz_max_20',
    'test00', 'test5_8', 'test10_10', 'test20_10',  'test20_20', 'test20_30', \
    'class0','sz_jt_10','sz_jt_20' ]
df_ds = df[col_list_ds].copy()
# 'test00', 'test5_8', 'test10_10', 'test20_10',  'test20_20', 'test20_30', \
# df_ds[ df_ds['test00'] ].shape, df_ds[ df_ds['test5_8'] ].shape, df_ds[ df_ds['test10_10'] ].shape, df_ds[df_ds['test20_10']].shape, df_ds[df_ds['test20_20']].shape, df_ds[ df_ds['test20_30'] ].shape

In [27]:
# df_ds[ df_ds['sz_jt_10']>=3 ]
# df_ds[ df_ds['class0']>=2 ]
# df_ds

In [30]:
""" 输出到mysql """
# 删除旧数据
sql = "truncate table ds_qf60"
basic.exec_sql(conn, sql)
# 输出新数据
df_ds.to_sql('ds_qf60', con=conn, index=False, if_exists='append', chunksize=2000) 

35170

In [25]:
# name: 数据库中的表名
# con: 与read_sql中相同，数据库连接的驱动
# if_exits： 当数据库中的这个表存在的时候，采取的措施是什么，包括三个值，默认为fail
# fail，若表存在，则不进行数据表写入的相关操作
# replace：若表存在，将数据库表中的数据覆盖；
# append：若表存在，将数据写到原表的后面。
# index：是否将df的index单独写到一列中
# index_label:指定列作为df的index输出，此时index为True
# dtype: 指定列的输出到数据库中的数据类型。字典形式储存：{column_name: sql_dtype}